In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import gc
gc.collect()

0

In [4]:
import pandas as pd
from src.utils import filtrar_meses, verificar_columnas_y_tipos, estandarizar_nombres_columnas
from src.utils import funciones 

In [5]:
test = pd.read_csv("data/test/test.csv")
test.to_csv("data/test/test_pr.csv", index=False)

In [6]:
import geopandas as gpd

barrios_gdf = gpd.read_file("data/barrios/barrios.shp")

# Ver columnas reales
print("🧩 Columnas disponibles en el shapefile:")
print(barrios_gdf.columns)

🧩 Columnas disponibles en el shapefile:
Index(['id', 'objeto', 'nombre', 'comuna', 'perimetro_', 'area_metro',
       'geometry'],
      dtype='object')


In [7]:
import pandas as pd
import os

# Rutas de los datasets en orden descendente
archivos = [
    "data/test/test_pr.csv",
]

# Diccionario de estaciones únicas: id_estacion -> (lat, lon)
estaciones_dict = {}

for path in archivos:
    if not os.path.exists(path):
        print(f"⚠️ Archivo no encontrado: {path}")
        continue

    df = pd.read_csv(path)

    for rol in ["origen", "destino"]:
        for _, row in df.iterrows():
            est_id = row[f"id_estacion_{rol}"]
            lat = row[f"lat_estacion_{rol}"]
            lon = row[f"long_estacion_{rol}"]

            if pd.notna(est_id) and pd.notna(lat) and pd.notna(lon):
                if est_id not in estaciones_dict:
                    estaciones_dict[est_id] = (lat, lon)
        

In [8]:
df_estaciones = pd.DataFrame([
    {"id_estacion": est_id, "lat": lat, "lon": lon}
    for est_id, (lat, lon) in estaciones_dict.items()
])

In [9]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Cargar shapefile de barrios
barrios_gdf = gpd.read_file("data/barrios/barrios.shp")

if "nombre" in barrios_gdf.columns and "barrio" not in barrios_gdf.columns:
    barrios_gdf = barrios_gdf.rename(columns={"nombre": "barrio"})

# Crear GeoDataFrame desde df_estaciones (ya lo tenés construido)
df_estaciones["geometry"] = df_estaciones.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
estaciones_gdf = gpd.GeoDataFrame(df_estaciones, geometry="geometry", crs=barrios_gdf.crs)

# Join espacial (punto dentro de polígono)
estaciones_con_barrios = gpd.sjoin(estaciones_gdf, barrios_gdf, how="left", predicate="within")

# Crear CSV (opcional)
estaciones_con_barrios[["id_estacion", "lat", "lon", "barrio"]].to_csv("data/estaciones_con_barrios.csv", index=False)

# Crear diccionario: id_estacion -> barrio
mapa_barrio = estaciones_con_barrios.set_index("id_estacion")["barrio"].to_dict()

# Sobrescribir manualmente los valores deseados
mapa_barrio[111] = "PUERTO MADERO"
mapa_barrio[541] = "PALERMO"

path = "data/test/test_pr.csv"
df = pd.read_csv(path)
df["barrio_origen"] = df["id_estacion_origen"].map(mapa_barrio)
df["barrio_destino"] = df["id_estacion_destino"].map(mapa_barrio)
df.to_csv(path, index=False)
print("✅ Enriquecido con barrios: test")

✅ Enriquecido con barrios: test


In [10]:
import pandas as pd
import os

# Años relevantes
anios_recorridos = [2020, 2021, 2022, 2023, 2024]
anios_usuarios_nuevos = [2015, 2016, 2017, 2018, 2019]

# === 1. Cargar todos los usuarios que aparecen en recorridos ===
usuarios_recorridos = []

path_test = "data/test/test_pr.csv"
if os.path.exists(path_test):
    df_test = pd.read_csv(path_test, usecols=["id_usuario"])
    df_test = df_test.dropna().astype({"id_usuario": int})
    df_test["año_recorrido"] = 2024  # Asignar año de test
    usuarios_recorridos.append(df_test)
else:
    print("⚠️ No se encontró archivo de test")

df_usuarios_recorridos = pd.concat(usuarios_recorridos, ignore_index=True)


usuarios_registrados = []
path = f"data/usuarios/processed/usuarios_ecobici_2024_limpio.csv"
if os.path.exists(path):
    df = pd.read_csv(path, usecols=["ID_usuario"])
    df = df.dropna().astype({"ID_usuario": int})
    usuarios_registrados.append(df)
else:
    print(f"⚠️ No se encontró archivo de usuarios año")


df_usuarios_registrados = pd.concat(usuarios_registrados, ignore_index=True)
usuarios_2020_2024_set = set(df_usuarios_registrados["ID_usuario"].unique())


df_no_registrados = df_usuarios_recorridos[
    ~df_usuarios_recorridos["id_usuario"].isin(usuarios_2020_2024_set)
].copy()


In [11]:
import pandas as pd
import os

# Cargar viajes
df_viajes = pd.read_csv("data/test/test_pr.csv")

data_usuario = pd.read_csv("data/usuarios/processed/usuarios_ecobici_2024_limpio.csv")
df_filtrado = data_usuario[data_usuario["mes_alta"] > 8].copy()
df_filtrado.to_csv("data/usuarios/processed/usuarios_ecobici_2024_limpio_filtrado_test.csv", index=False)
df_usuarios = pd.read_csv("data/usuarios/processed/usuarios_ecobici_2024_limpio_filtrado_test.csv")  # debe tener id_usuario, mes_alta, anio_alta, fecha_alta


df_estaciones = pd.read_csv("data/estaciones_con_barrios.csv")  # debe tener id_estacion, barrio, lat, lon
from modeling import construir_dataset_modelado_v2

df_modelado = construir_dataset_modelado_v2(df_viajes, df_usuarios, df_estaciones)
df_modelado.to_csv("data/modelado/ds_modelado_test.csv", index=False)


c:\Users\catal\OneDrive\Documents\UDESA\AñoIII\Machine Learning\Proyecto_Final\FINAL_ML\modeling.py:22: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta)
c:\Users\catal\OneDrive\Documents\UDESA\AñoIII\Machine Learning\Proyecto_Final\FINAL_ML\modeling.py:22: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta)
c:\Users\catal\OneDrive\Documents\UDESA\AñoIII\Machine Learning\Proyecto_Final\FINAL_ML\modeling.py:22: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta)
Contando estaciones cercanas: 100%|██████████| 398/398 [00:11<00:00, 35.46it/s]


In [12]:
df_modelado = pd.read_csv("data/modelado/ds_modelado_test.csv")

In [13]:
print(df_modelado.columns)

Index(['id_recorrido', 'duracion_recorrido', 'fecha_origen_recorrido',
       'id_estacion_origen', 'fecha_destino_recorrido', 'id_estacion_destino',
       'id_usuario', 'modelo_bicicleta', 'barrio_origen', 'barrio_destino',
       'fecha_intervalo', 'hora_dia', 'dia_semana', 'es_finde',
       'estacion_del_anio', 'edad_usuario', 'año_alta', 'mes_alta',
       'genero_FEMALE', 'genero_MALE', 'genero_OTHER', 'usuario_registrado',
       'zona_destino_cluster', 'zona_origen_cluster',
       'cantidad_estaciones_cercanas_destino',
       'cantidad_estaciones_cercanas_origen', 'N_arribos_intervalo',
       'N_salidas_intervalo'],
      dtype='object')


In [20]:
mapa_estaciones = {
    "verano": 1,
    "otono": 2,
    "invierno": 3,
    "primavera": 4
}

df_modelado["estacion_del_anio"] = df_modelado["estacion_del_anio"].map(mapa_estaciones).astype(int)
df_modelado["fecha_origen_recorrido"] = pd.to_datetime(df_modelado["fecha_origen_recorrido"], errors="coerce")
df_modelado["fecha_destino_recorrido"] = pd.to_datetime(df_modelado["fecha_destino_recorrido"], errors="coerce")
df_modelado["fecha_intervalo"] = pd.to_datetime(df_modelado["fecha_intervalo"], errors="coerce")

df_modelado["año_origen"] = df_modelado["fecha_origen_recorrido"].dt.year
df_modelado["mes_origen"] = df_modelado["fecha_origen_recorrido"].dt.month
df_modelado["dia_origen"] = df_modelado["fecha_origen_recorrido"].dt.day
df_modelado["hora_origen"] = df_modelado["fecha_origen_recorrido"].dt.hour
df_modelado["minuto_origen"] = df_modelado["fecha_origen_recorrido"].dt.minute
df_modelado["segundo_origen"] = df_modelado["fecha_origen_recorrido"].dt.second

df_modelado["año_destino"] = df_modelado["fecha_destino_recorrido"].dt.year
df_modelado["mes_destino"] = df_modelado["fecha_destino_recorrido"].dt.month
df_modelado["dia_destino"] = df_modelado["fecha_destino_recorrido"].dt.day
df_modelado["hora_destino"] = df_modelado["fecha_destino_recorrido"].dt.hour
df_modelado["minuto_destino"] = df_modelado["fecha_destino_recorrido"].dt.minute
df_modelado["segundo_destino"] = df_modelado["fecha_destino_recorrido"].dt.second

df_modelado["año_intervalo"] = df_modelado["fecha_intervalo"].dt.year
df_modelado["mes_intervalo"] = df_modelado["fecha_intervalo"].dt.month
df_modelado["dia_intervalo"] = df_modelado["fecha_intervalo"].dt.day
df_modelado["hora_intervalo"] = df_modelado["fecha_intervalo"].dt.hour
df_modelado["minuto_intervalo"] = df_modelado["fecha_intervalo"].dt.minute

df_modelado["edad_usuario"] = pd.to_numeric(df_modelado["edad_usuario"], errors="coerce").fillna(-1).astype(int)

barrios_unicos = pd.unique(df_modelado[["barrio_origen", "barrio_destino"]].values.ravel())
mapa_barrio = {barrio: i for i, barrio in enumerate(barrios_unicos, start=1)}

df_modelado["barrio_origen"] = df_modelado["barrio_origen"].map(mapa_barrio)
df_modelado["barrio_destino"] = df_modelado["barrio_destino"].map(mapa_barrio)
df_modelado["modelo_bicicleta"] = df_modelado["modelo_bicicleta"].map({"ICONIC": 1, "FIT": 0}).astype(int)

df_modelado.drop(columns=["hora_dia"], inplace=True)
df_modelado.drop(columns=["fecha_origen_recorrido", "fecha_destino_recorrido", "fecha_intervalo"], inplace=True)

df_modelado.to_csv("data/modelado/ds_modelado_test.csv", index=False)

In [21]:
df_modelado = pd.read_csv("data/modelado/ds_modelado_test.csv")
print(df_modelado.isnull().sum())

id_recorrido                               0
duracion_recorrido                         0
id_estacion_origen                         0
id_estacion_destino                        0
id_usuario                                 0
modelo_bicicleta                           0
barrio_origen                              0
barrio_destino                             0
dia_semana                                 0
es_finde                                   0
estacion_del_anio                          0
edad_usuario                               0
año_alta                                   0
mes_alta                                   0
genero_FEMALE                              0
genero_MALE                                0
genero_OTHER                               0
usuario_registrado                         0
zona_destino_cluster                       0
zona_origen_cluster                        0
cantidad_estaciones_cercanas_destino       0
cantidad_estaciones_cercanas_origen        0
N_arribos_

In [28]:
import pandas as pd

# Crear columna datetime de origen
df_modelado["fecha_origen"] = pd.to_datetime(dict(
    year=df_modelado["año_origen"],
    month=df_modelado["mes_origen"],
    day=df_modelado["dia_origen"],
    hour=df_modelado["hora_origen"],
    minute=df_modelado["minuto_origen"],
    second=df_modelado["segundo_origen"]
), errors="coerce")  # por si hay algo raro

# Sumar duración en segundos para obtener fecha destino
df_modelado["fecha_destino"] = df_modelado["fecha_origen"] + pd.to_timedelta(df_modelado["duracion_recorrido"], unit="s")

# Extraer nueva info de fecha/hora de destino
df_modelado["año_destino"] = df_modelado["fecha_destino"].dt.year
df_modelado["mes_destino"] = df_modelado["fecha_destino"].dt.month
df_modelado["dia_destino"] = df_modelado["fecha_destino"].dt.day
df_modelado["hora_destino"] = df_modelado["fecha_destino"].dt.hour
df_modelado["minuto_destino"] = df_modelado["fecha_destino"].dt.minute
df_modelado["segundo_destino"] = df_modelado["fecha_destino"].dt.second

# Verificar que no queden nulos
print(df_modelado[["año_destino", "mes_destino", "dia_destino", "hora_destino", "minuto_destino", "segundo_destino"]].isna().sum())

# Eliminar columnas auxiliares si no las querés
df_modelado = df_modelado.drop(columns=["fecha_origen", "fecha_destino"])
df_modelado.to_csv("data/modelado/ds_modelado_test.csv", index=False)


año_destino        0
mes_destino        0
dia_destino        0
hora_destino       0
minuto_destino     0
segundo_destino    0
dtype: int64


In [29]:
df = pd.read_csv("data/modelado/ds_modelado_test.csv")

# Crear la columna de fecha/hora para orden temporal
df["fecha_intervalo"] = pd.to_datetime(dict(
    year=df["año_intervalo"],
    month=df["mes_intervalo"],
    day=df["dia_intervalo"],
    hour=df["hora_intervalo"],
    minute=df["minuto_intervalo"]
))

# Ordenar por estación y tiempo
df = df.sort_values(by=["id_estacion_origen", "fecha_intervalo"]).copy()

# Agregar rolling promedio (solo usando datos anteriores, no incluye el actual)
df["N_SALIDAS_PROM_2INT"] = (
    df.groupby("id_estacion_origen")["N_salidas_intervalo"]
      .shift(1)
      .rolling(window=2)
      .mean()
      .reset_index(drop=True)
)

df["N_ARRIBOS_PROM_2INT"] = (
    df.groupby("id_estacion_origen")["N_arribos_intervalo"]
      .shift(1)
      .rolling(window=2)
      .mean()
      .reset_index(drop=True)
)

# Nuevas: lags de 1, 2 y 3 para salidas y arribos
for lag in [1, 2, 3]:
    df[f"N_SALIDAS_LAG{lag}"] = (
        df.groupby("id_estacion_origen")["N_salidas_intervalo"].shift(lag)
    )
    df[f"N_ARRIBOS_LAG{lag}"] = (
        df.groupby("id_estacion_origen")["N_arribos_intervalo"].shift(lag)
    )

# Nuevas: lags para columnas que terminan en 'destino'
columnas_destino = [col for col in df.columns if col.endswith("destino")]
for col in columnas_destino:
    for lag in [1, 2, 3]:
        df[f"{col}_LAG{lag}"] = df.groupby("id_estacion_destino")[col].shift(lag)


df.to_csv("data/modelado/ds_modelado_test.csv", index=False)

In [30]:
df = pd.read_csv("data/modelado/ds_modelado_test.csv")
print(df.isna().sum()[df.isna().sum() > 0])

N_SALIDAS_PROM_2INT                           790
N_ARRIBOS_PROM_2INT                           790
N_SALIDAS_LAG1                                395
N_ARRIBOS_LAG1                                395
N_SALIDAS_LAG2                                790
N_ARRIBOS_LAG2                                790
N_SALIDAS_LAG3                               1185
N_ARRIBOS_LAG3                               1185
id_estacion_destino_LAG1                      398
id_estacion_destino_LAG2                      796
id_estacion_destino_LAG3                     1193
barrio_destino_LAG1                           398
barrio_destino_LAG2                           796
barrio_destino_LAG3                          1193
cantidad_estaciones_cercanas_destino_LAG1     398
cantidad_estaciones_cercanas_destino_LAG2     796
cantidad_estaciones_cercanas_destino_LAG3    1193
año_destino_LAG1                              398
año_destino_LAG2                              796
año_destino_LAG3                             1193


In [36]:
#mes origen mayor a 8 y que sean nans
df = df[df["mes_origen"] > 8].copy()
print(df.isna().sum()[df.isna().sum() > 0])
    

N_SALIDAS_PROM_2INT                          286
N_ARRIBOS_PROM_2INT                          286
N_SALIDAS_LAG1                                21
N_ARRIBOS_LAG1                                21
N_SALIDAS_LAG2                                42
N_ARRIBOS_LAG2                                42
N_SALIDAS_LAG3                                63
N_ARRIBOS_LAG3                                63
id_estacion_destino_LAG1                      91
id_estacion_destino_LAG2                     182
id_estacion_destino_LAG3                     278
barrio_destino_LAG1                           91
barrio_destino_LAG2                          182
barrio_destino_LAG3                          278
cantidad_estaciones_cercanas_destino_LAG1     91
cantidad_estaciones_cercanas_destino_LAG2    182
cantidad_estaciones_cercanas_destino_LAG3    278
año_destino_LAG1                              91
año_destino_LAG2                             182
año_destino_LAG3                             278
mes_destino_LAG1    

In [38]:
df.to_csv("data/modelado/ds_modelado_test.csv", index=False)

In [43]:
# --- Agrupar columnas según tipo de imputación adecuada ---
cols_promedios = [col for col in df.columns if "PROM" in col or "LAG" in col and "N_" in col]
# Para los primeros valores, se llena de 0, ya que son promedios o conteos
df[cols_promedios] = df[cols_promedios].fillna(0)



cols_id = [col for col in df.columns if col.startswith("id_") and "LAG" in col]
cols_tiempo = [col for col in df.columns if any(t in col for t in ['año_', 'mes_', 'dia_', 'hora_', 'minuto_', 'segundo_']) and 'LAG' in col]
cols_barrio = [col for col in df.columns if "barrio" in col and "LAG" in col]
cols_estaciones = [col for col in df.columns if "cantidad_estaciones" in col and "LAG" in col]
#como son categorías, se llenan con -1
#en vez de poner 0 que representa una categoría válida, se usa -1 para indicar que no hay información

df[cols_id + cols_barrio + cols_tiempo + cols_estaciones] = df[cols_id + cols_barrio + cols_tiempo + cols_estaciones].fillna(-1)

In [44]:
df = pd.read_csv("data/modelado/ds_modelado_test.csv")
df = df[df["mes_origen"] > 8].copy()
print(df.isna().sum()[df.isna().sum() > 0])

Series([], dtype: int64)


In [45]:
import pandas as pd

# === Cargar dataset ===
df = pd.read_csv("data/modelado/ds_modelado_test.csv", parse_dates=["fecha_intervalo"])

# === Crear columna de referencia (la estación a la que querés predecir arribos) ===
df["estacion_referencia"] = df["id_estacion_destino"]

# === Eliminar columnas anteriores de arribos, salidas y lags ===
cols_a_eliminar = [col for col in df.columns if (
    col.startswith("N_ARRIBOS_LAG") or 
    col.startswith("N_SALIDAS_LAG") or 
    col in ["N_arribos_intervalo", "N_salidas_intervalo"]
)]
df = df.drop(columns=cols_a_eliminar)

# === Recalcular N_arribos_intervalo ===
arribos = (
    df.groupby(["estacion_referencia", "fecha_intervalo"])
    .size()
    .rename("N_arribos_intervalo")
    .reset_index()
)

# === Recalcular N_salidas_intervalo (asociadas a la estación_referencia) ===
salidas = (
    df[df["id_estacion_origen"].notna()]
    .groupby(["id_estacion_origen", "fecha_intervalo"])
    .size()
    .rename("N_salidas_intervalo")
    .reset_index()
    .rename(columns={"id_estacion_origen": "estacion_referencia"})
)

# === Merge con el dataset original ===
df = df.merge(arribos, on=["estacion_referencia", "fecha_intervalo"], how="left")
df = df.merge(salidas, on=["estacion_referencia", "fecha_intervalo"], how="left")

# === Completar con ceros donde no hubo actividad ===
df["N_arribos_intervalo"] = df["N_arribos_intervalo"].fillna(0).astype(int)
df["N_salidas_intervalo"] = df["N_salidas_intervalo"].fillna(0).astype(int)

# === Recalcular lags respecto a estacion_referencia ===
df = df.sort_values(["estacion_referencia", "fecha_intervalo"])

for lag in [1, 2, 3, 4, 5, 6]:
    df[f"N_ARRIBOS_LAG{lag}"] = (
        df.groupby("estacion_referencia")["N_arribos_intervalo"].shift(lag)
    )
    df[f"N_SALIDAS_LAG{lag}"] = (
        df.groupby("estacion_referencia")["N_salidas_intervalo"].shift(lag)
    )
df.to_csv("data/modelado/ds_modelado_test_ref.csv", index=False)

In [51]:
df = pd.read_csv("data/modelado/ds_modelado_test_ref.csv")
print(df.isna().sum()[df.isna().sum() > 0])

N_ARRIBOS_LAG1     394
N_SALIDAS_LAG1     394
N_ARRIBOS_LAG2     787
N_SALIDAS_LAG2     787
N_ARRIBOS_LAG3    1180
N_SALIDAS_LAG3    1180
N_ARRIBOS_LAG4    1573
N_SALIDAS_LAG4    1573
N_ARRIBOS_LAG5    1966
N_SALIDAS_LAG5    1966
N_ARRIBOS_LAG6    2359
N_SALIDAS_LAG6    2359
dtype: int64


In [52]:
# --- Agrupar columnas según tipo de imputación adecuada ---
cols_promedios = [col for col in df.columns if "PROM" in col or "LAG" in col and "N_" in col]
# Para los primeros valores, se llena de 0, ya que son promedios o conteos
df[cols_promedios] = df[cols_promedios].fillna(0)



cols_id = [col for col in df.columns if col.startswith("id_") and "LAG" in col]
cols_tiempo = [col for col in df.columns if any(t in col for t in ['año_', 'mes_', 'dia_', 'hora_', 'minuto_', 'segundo_']) and 'LAG' in col]
cols_barrio = [col for col in df.columns if "barrio" in col and "LAG" in col]
cols_estaciones = [col for col in df.columns if "cantidad_estaciones" in col and "LAG" in col]
#como son categorías, se llenan con -1
#en vez de poner 0 que representa una categoría válida, se usa -1 para indicar que no hay información

df[cols_id + cols_barrio + cols_tiempo + cols_estaciones] = df[cols_id + cols_barrio + cols_tiempo + cols_estaciones].fillna(-1)

In [53]:
df.to_csv("data/modelado/ds_modelado_test_ref.csv", index=False)

In [54]:
df = pd.read_csv("data/modelado/ds_modelado_test_ref.csv")
print(df.isna().sum()[df.isna().sum() > 0])

Series([], dtype: int64)


In [55]:
# Filtrar solo los meses de septiembre a diciembre
df_fin = df[df["mes_origen"].isin([9, 10, 11, 12])].copy()
# Guardar CSV
df_fin.to_csv("data/modelado/ds_modelado_test_ref.csv", index=False)

In [56]:
df_fin = pd.read_csv("data/modelado/ds_modelado_test_ref.csv")
print(df_fin.isna().sum()[df.isna().sum() > 0])

Series([], dtype: int64)
